## Data processing and Analysis

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math

In [ ]:
buggy_elements_df = pd.read_csv('../data/generated/buggy-code-lifetime-data.csv', sep=',')

In [ ]:
buggy_elements_df.head()

## Collecting lifetime grouping by bug_id,buggy_line and buggy_component

In [ ]:
groupby_author_commit_date_dict =  buggy_elements_df.groupby(['bug_id','buggy_line_number','buggy_component','bug_type','fix_commit_hash'])['author_commit_date'].apply(list).to_dict()

In [ ]:
print(groupby_author_commit_date_dict)

In [ ]:
for k, v in groupby_author_commit_date_dict.items():
    
    v.sort()
    lifetime = v[-1] - v[0]
    groupby_author_commit_date_dict[k] = lifetime

In [ ]:
print(groupby_author_commit_date_dict)

## Parsing dict to obtain lifetime per component into a dataframe

In [ ]:
data_items = groupby_author_commit_date_dict.items()
data_list = list(data_items)

lifetime_per_component_df = pd.DataFrame(data_list)

In [ ]:
lifetime_per_component_df.head()

In [ ]:
def obtain_fifth_element(x):

    return x[4]

In [ ]:
def obtain_fourth_element(x):

    return x[3]

In [ ]:
def obtain_third_element(x):

    return x[2]

In [ ]:
def obtain_second_element(x):

    return x[1]

In [ ]:
def obtain_first_element(x):

    return x[0]

In [ ]:
lifetime_per_component_df['Fix_Commit_Hash'] = lifetime_per_component_df[0].apply(obtain_fifth_element)

In [ ]:
lifetime_per_component_df['Bug Type'] = lifetime_per_component_df[0].apply(obtain_fourth_element)

In [ ]:
lifetime_per_component_df['Buggy Line'] = lifetime_per_component_df[0].apply(obtain_second_element)

In [ ]:
lifetime_per_component_df['Bug ID'] = lifetime_per_component_df[0].apply(obtain_first_element)

In [ ]:
lifetime_per_component_df[0] = lifetime_per_component_df[0].apply(obtain_third_element)

In [ ]:
lifetime_per_component_df.rename(columns = { 0 : 'Buggy Component', 1 : 'Lifetime'}, inplace = True)

In [ ]:
lifetime_per_component_df.head()

## Collecting Number of Authors and Number of times modified grouping by bug_id,buggy_line and buggy_component

In [ ]:
groupby_author_dict =  buggy_elements_df.groupby(['bug_id','buggy_line_number','buggy_component','bug_type','fix_commit_hash'])['author_name'].apply(list).to_dict()

In [ ]:
print(groupby_author_dict)

In [ ]:
Number_Authors = []
Number_Modified = []

In [ ]:
for k, v in groupby_author_dict.items():
    
    x = np.array(v)
    Number_Modified.append(len(x))
    Number_Authors.append(len(np.unique(x)))
    

# Adding number of authors to dataframe

In [ ]:
lifetime_per_component_df['Number of Authors'] = Number_Authors

# Adding number of times modified to dataframe

In [ ]:
lifetime_per_component_df['Number of Times Modified'] = Number_Modified

In [ ]:
lifetime_per_component_df.head()

In [ ]:
lifetime_per_component_df.shape

# GroupBy buggy component and aggreation for Lifetime, Number of Authors and Number of times modified using the mean as metric

In [ ]:
component_df = lifetime_per_component_df.groupby('Buggy Component').aggregate({'Lifetime': 'mean',
                             'Number of Authors': 'mean','Number of Times Modified' : 'mean'})


# Converting seconds to days 

In [ ]:
def second_to_days(x):
    
    return x/60/60/24

In [ ]:
## Testing function
## input : 14322352 seconds
## expected output : ~165.76796296 days

print(second_to_days(14322352))


In [ ]:
component_df['Lifetime'] = component_df['Lifetime'].apply(second_to_days)

In [ ]:
component_df.head()

## Overall Number of Author, Number of Times modified, Lifetime buggy code element distribution

In [ ]:


ax = component_df.plot.barh(rot=0, figsize=(25,30), color={"Lifetime": "lightgray", "Number of Authors": "gray", 'Number of Times Modified' : 'black'})

for p in ax.patches:
    
    ax.annotate(str(p.get_width()), (p.get_x() + p.get_width(), p.get_y()), xytext=(2, 3), textcoords='offset points', size = 6)

In [ ]:
ax.figure.savefig("RQ2_Average_Lifetime_NAuthors_NModified.pdf", bbox_inches='tight',)

## Quantum Number of Author, Number of Times modified, Lifetime buggy code element distribution

In [ ]:
quantum_bug_dataframe = lifetime_per_component_df.loc[((lifetime_per_component_df['Bug Type'] == 'Quantum'))]

In [ ]:
quantum_component_df = quantum_bug_dataframe.groupby('Buggy Component').aggregate({'Lifetime': 'mean',
                             'Number of Authors': 'mean','Number of Times Modified' : 'mean'})

In [ ]:
quantum_component_df['Lifetime'] = quantum_component_df['Lifetime'].apply(second_to_days)

In [ ]:
quantum_ax = quantum_component_df.plot.barh(rot=0, figsize=(25,25), color={"Lifetime": "lightgray", "Number of Authors": "gray", 'Number of Times Modified' : 'black'})

for p in quantum_ax.patches:
    
    quantum_ax.annotate(str(p.get_width()), (p.get_x() + p.get_width(), p.get_y()), xytext=(2, 3), textcoords='offset points', size = 6)

In [ ]:
quantum_ax.figure.savefig("RQ2_Average_Lifetime_NAuthors_NModified_Quantum.pdf", bbox_inches='tight',)

## Classical Number of Author, Number of Times modified, Lifetime buggy code element distribution

In [ ]:
classical_bug_dataframe = lifetime_per_component_df.loc[((lifetime_per_component_df['Bug Type'] == 'Classical'))]

In [ ]:
classical_component_df = classical_bug_dataframe.groupby('Buggy Component').aggregate({'Lifetime': 'mean',
                             'Number of Authors': 'mean','Number of Times Modified' : 'mean'})

In [ ]:
classical_component_df['Lifetime'] = classical_component_df['Lifetime'].apply(second_to_days)

In [ ]:
classical_ax = classical_component_df.plot.barh(rot=0, figsize=(25,25), color={"Lifetime": "lightgray", "Number of Authors": "gray", 'Number of Times Modified' : 'black'})

for p in classical_ax.patches:
    
    classical_ax.annotate(str(p.get_width()), (p.get_x() + p.get_width(), p.get_y()), xytext=(2, 3), textcoords='offset points', size = 6)

In [ ]:
classical_ax.figure.savefig("RQ2_Average_Lifetime_NAuthors_NModified_Classical.pdf", bbox_inches='tight',)

## Boxplot

In [ ]:
classical_boxplot_component_bugId_df = classical_bug_dataframe.groupby(['Buggy Component', 'Bug ID']).aggregate({'Lifetime': 'mean',
                             'Number of Authors': 'mean','Number of Times Modified' : 'mean'})

In [ ]:
classical_boxplot_component_bugId_df['Lifetime'] = classical_boxplot_component_bugId_df['Lifetime'].apply(second_to_days)

In [ ]:
classical_boxplot_component_bugId_df.head()

In [ ]:
quantum_boxplot_component_bugId_df =  quantum_bug_dataframe.groupby(['Buggy Component', 'Bug ID']).aggregate({'Lifetime': 'mean',
                             'Number of Authors': 'mean','Number of Times Modified' : 'mean'})

In [ ]:
quantum_boxplot_component_bugId_df['Lifetime'] = quantum_boxplot_component_bugId_df['Lifetime'].apply(second_to_days)

In [ ]:
quantum_boxplot_component_bugId_df.head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize = (25,25))

classical_boxplot_component_bugId_df.boxplot(column=['Lifetime'],by='Buggy Component',vert=False, figsize=(25,25),ax = ax2)

quantum_boxplot_component_bugId_df.boxplot(column=['Lifetime'],by='Buggy Component',vert=False, figsize=(25,25), ax = ax1)

plt.title('')
plt.suptitle('')
ax1.set_title('Quantum');
ax1.set_xlabel("");
ax1.set_ylabel("");
ax2.set_title('Classical');
ax2.set_xlabel("");
ax2.set_ylabel("");

fig.savefig("Average_Lifetime_Distribution_Per_BugID_BuggyComponent.pdf", bbox_inches='tight',)




In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize = (25,25))

classical_boxplot_component_bugId_df.boxplot(column=['Number of Times Modified'],by='Buggy Component',vert=False, figsize=(25,25),ax = ax2)

quantum_boxplot_component_bugId_df.boxplot(column=['Number of Times Modified'],by='Buggy Component',vert=False, figsize=(25,25), ax = ax1)

plt.title('')
plt.suptitle('')
ax1.set_title('Quantum');
ax1.set_xlabel("");
ax1.set_ylabel("");
ax2.set_title('Classical');
ax2.set_xlabel("");
ax2.set_ylabel("");


fig.savefig("Average_NumberOfTimesModified_Distribution_Per_BugID_BuggyComponent.pdf", bbox_inches='tight',)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize = (25,25))

classical_boxplot_component_bugId_df.boxplot(column=['Number of Authors'],by='Buggy Component',vert=False, figsize=(25,25),ax = ax2)

quantum_boxplot_component_bugId_df.boxplot(column=['Number of Authors'],by='Buggy Component',vert=False, figsize=(25,25), ax = ax1)

plt.title('')
plt.suptitle('')
ax1.set_title('Quantum');
ax1.set_xlabel("");
ax1.set_ylabel("");
ax2.set_title('Classical');
ax2.set_xlabel("");
ax2.set_ylabel("");



fig.savefig("Average_NumberOfAuthors_Distribution_Per_BugID_BuggyComponent.pdf", bbox_inches='tight',)